# Helpers borrowed from [Peter Norvig](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb)

In [1]:
import re
import math
from collections import OrderedDict, Counter, defaultdict, namedtuple, deque

from itertools   import permutations, combinations, chain, takewhile, \
cycle, product, islice, zip_longest, count as count_from

def Input(day):
    "Open this day's input file."
    filename = 'input_day{}.txt'.format(day)
    return open(filename).read().strip()

cat = ''.join

def array(lines):
    "Convert an iterable of lines into a 2-D array. If lines is a str, do splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return [mapt(atom, line.split()) for line in lines]

def atom(token):
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)
            
def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def X(point): return point[0]
def Y(point): return point[1]

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))


# [Day 1](https://adventofcode.com/2017/day/1)

In [2]:
def captcha(input):
    ints = mapt(int, input)
    left = (2*ints)[1:][:len(ints)]
    pairs = zip(ints, left)
    same = filter(lambda p: p[0] == p[1], pairs)
    return sum(map(lambda p: p[0], same))

assert captcha('1122') == 3
assert captcha('1111') == 4
assert captcha('1234') == 0
assert captcha('91212129') == 9

captcha(Input(1))

1144

In [3]:
def captcha_half(input):
    ints = mapt(int, input)
    left = (2*ints)[int(len(ints)/2):][:len(ints)]
    pairs = zip(ints, left)
    same = filter(lambda p: p[0] == p[1], pairs)
    return sum(map(lambda p: p[0], same))

assert captcha_half('1212') == 6
assert captcha_half('1221') == 0
assert captcha_half('123425') == 4
assert captcha_half('123123') == 12
assert captcha_half('12131415') == 4

captcha_half(Input(1))

1194

# [Day 2](https://adventofcode.com/2017/day/2)

In [4]:
def spreadsheet_checksum(input):
    return sum([max(row) - min(row)
                for row in array(input)
               ])

spreadsheet_checksum(Input(2))

45351

In [5]:
def spreadsheet_checksum2(input):
    return sum([p[0] // p[1]
         for row in array(input)
         for p in permutations(row, 2) 
         if p[0] > p[1] 
         and p[0] % p[1] == 0])
        
spreadsheet_checksum2(Input(2))

275

# [Day 3](https://adventofcode.com/2017/day/3)

In [6]:
position = 277678

a = math.ceil((math.sqrt(position)))
if a % 2 == 0:
    a += 1
s = a * a - 4 * (a - 1) + 1
h = (a - 1) // 2
d = abs(((position - s + 1) % (a - 1)) - h)
h + d

475

In [7]:
def populate(m, p, f):
    cx, cy = p
    px = cx - 1
    py = cy
    
    while (cx - 1, cy) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cy += 1
        
    while (cx, cy - 1) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cx -= 1
        
    while (cx + 1, cy) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cy -= 1
        
    while (cx, cy + 1) in m:
        m[(cx, cy)] = f(m, (px, py), (cx, cy))
        px = cx
        py = cy
        cx += 1
        
    return (cx, cy)

def mem_find(cell):
    n = (1, 0)
    mem = OrderedDict([((0, 0), 1)])
    
    while not cell in mem.values():
        n = populate(mem, n, lambda m, p, c: m[p] + 1)
        
    for p, c in mem.items():
        if c == cell:
            return p

p = mem_find(position)
cityblock_distance(p)


475

In [8]:
def neigh(m, p, c):
    return sum([m[n] for n in neighbors8(c) if n in m])
        
def mem_neigh_find(cell):
    n = (1, 0)
    mem = OrderedDict([((0, 0), 1)])
    
    while True:
        n = populate(mem, n, neigh)
        for p, c in mem.items():
            if c > cell:
                return c
        
mem_neigh_find(position)

279138

Different imlpementation based on complex numbers and a generator


In [9]:
U, D, R, L = 1j, -1j, 1, -1
turns = {R:U, U:L, L:D, D:R}

Cell = namedtuple('Cell', 'pos, dir, value')

def distance(point): 
    return abs(point.real) + abs(point.imag)

def cells(score):
    pos = 0
    direction = D
    v = 1
    occupied = {pos: v}
    
    while True:
        yield Cell(pos, direction, v)
        if not pos + turns[direction] in occupied:
            direction = turns[direction]
        
        pos += direction
        v = score(pos, v, occupied)
        occupied[pos] = v
            
def mem_find2():            
    for c in cells(lambda p, v, o: v + 1):
        if(c.value >= position):
            return int(distance(c.pos))

mem_find2()

475

In [10]:
def neigh8_c(p):
    return (p+R, p+L, p+U, p+D, p+R+U, p+R+D, p+L+U, p+L+D)

def mem_neigh_find2():            
    for c in cells(lambda p, v, o: sum([o[n] for n in neigh8_c(p) if n in o])):
        if(c.value >= position):
            return c.value

mem_neigh_find2()

279138

# [Day 4](https://adventofcode.com/2017/day/4)

In [11]:
def no_duplicates(passphrases):
    valid = []
    for passphrase in passphrases:
        if len(passphrase) == len(set(passphrase)):
            valid.append(passphrase)
            
    return valid
    
valid = no_duplicates(array(Input(4)))
len(valid)

466

In [12]:
def no_anagrams(passphrases):
    valid = []
    for passphrase in passphrases:
        ordred = mapt(cat, mapt(sorted, passphrase))
        if len(ordred) == len(set(ordred)):
            valid.append(passphrase)
        
    return valid
    
ana = no_anagrams(valid) 
len(ana)

251

# [Day 5](https://adventofcode.com/2017/day/5)

In [13]:
jumps = list(map(int, Input(5).split()))
pos = 0
steps = 0

while(pos < len(jumps)):
    p = pos
    pos += jumps[pos]
    jumps[p] += 1
    steps += 1
    
steps

358131

In [17]:
jumps = list(map(int, Input(5).split()))
pos = 0
steps = 0

while(pos < len(jumps)):
    p = pos
    pos += jumps[pos]
    if jumps[p] > 2:
        jumps[p] -= 1
    else:
        jumps[p] += 1  
    steps += 1
    
steps

CPU times: user 10.5 s, sys: 0 ns, total: 10.5 s
Wall time: 10.6 s
